In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
#https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import train and test data and view data format


In [ ]:

data = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv')
data.head()

test_data = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/test.csv')

In [ ]:
print(type(data['pixels']))

In [ ]:
#create 3 channel numpy array of [num_examples, 3, 48, 48] 
#type for np.array dim 3 should be uint8 for conversion to pil image
train_imgs = []
test_imgs = []

for image in data['pixels']:
    img = image.split()
    img = np.array([float(i) for i in img], dtype=np.uint8)
    img = np.reshape(img, (48,48))
    img = np.array([img]*3)
    train_imgs.append(img)
train_imgs = np.array(train_imgs)
train_labels = [np.array([i]) for i in data['emotion'].to_numpy()]

for image in test_data['pixels']:
    img = image.split()
    img = np.array([float(i) for i in img], dtype=np.uint8)
    img = np.reshape(img, (48,48))
    img = np.array([img]*3)
    test_imgs.append(img)
test_imgs = np.array(test_imgs)

In [ ]:
print(train_labels)

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [ ]:
class FaceDataset(Dataset):
    """Custom data set for faces and labels input as numpy array"""
    def __init__(self, samples, labels=None, transform=None):

        self.samples = samples
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if self.labels == None:
            sample = {'image': torch.from_numpy(self.samples[idx])}
        else:
            sample = {'image': torch.from_numpy(self.samples[idx]), 'label': torch.from_numpy(self.labels[idx])}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [ ]:
#compose transforms and datasets

transform = transforms.Compose([transforms.ToPILImage(), 
                                transforms.Resize((224, 224)),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation(20),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

transform1 = transforms.Compose([transforms.ToPILImage(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

temp_dataset = FaceDataset(train_imgs, train_labels, transform=transform)

split = int(np.round(len(temp_dataset)*0.8))

train_dataset, val_dataset = torch.utils.data.random_split(temp_dataset, [split, len(temp_dataset) - split])
test_dataset = FaceDataset(test_imgs, transform=transform1)

train_loader = DataLoader(train_dataset, batch_size = 24, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 24, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 24, shuffle = True)

In [ ]:
print(next(iter(val_loader))['imag'].squeeze())

In [ ]:
import matplotlib.pyplot as plt

example = next(iter(train_dataset))

plt.imshow(example['image'])
print(example['label'])

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.resnet18(pretrained=True)
#print(resnet18)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(nn.Linear(512, 256), 
                        nn.ReLU(), 
                        nn.Linear(256, 100), 
                        nn.ReLU(),
                        nn.Linear(100, 7))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

#print(model)

In [ ]:
#training loop
epochs = 20
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Training on Device: {device}")
for epoch in range(epochs):
    running_loss = []
    
    model.train()
    model.to(device)
    n=0
    for item in train_loader:
        n+=1
        optimizer.zero_grad()
        
        image = item['image'].to(device)
        label = item['label'].to(device)
        label = label.squeeze()
        output = model(image)
        loss = criterion(output, label)
        
        loss.backward()
        if n%200 == 0:
            _, pred = torch.max(output.data, 1)
            #print(f"Output: {output}")
            #print(f"Label: {label}")
            #print(f"Pred: {pred}")
            
        running_loss.append(loss.item())
        
        optimizer.step()
        
        #validation loop
    with torch.no_grad():
        model.eval()
        total = 0
        correct = 0
        for item in val_loader:
            image = item['image'].to(device)
            label = item['label'].to(device)
            label = label.squeeze()
            output = model(image)
            _, pred = torch.max(output.data, 1)
            
            total+= label.size(0)
            correct += (pred == label).sum().item()
            
        print(f"Epoch: {epoch}")
        print(f"Training Loss: {sum(running_loss)/len(running_loss)}")
        print(f"Validation Accuracy: {float(correct)/total}")
            

In [ ]:
len(train_loader) + len(val_loader)